## Iran's Earthquakes Map

In [1]:
import xml.etree.ElementTree as ElementTree
import pandas as pd
import requests

In [2]:
# get 20 latest earthquackes reports
r = requests.get('http://irsc.ut.ac.ir/events_list.xml')

In [3]:
# Check if our request is OK
r.status_code

200

In [4]:
# Converting Eastern Arabic numbers to Western Arabic numbers - https://stackoverflow.com/a/14506662/846210
table = {1776: 48,  # 0 
         1777: 49,  # 1
         1778: 50,  # 2
         1779: 51,  # 3
         1780: 52,  # 4
         1781: 53,  # 5
         1782: 54,  # 6
         1783: 55,  # 7
         1784: 56,  # 8
         1785: 57}  # 9

def translate_to_western(x):
    return x.translate(table)

In [5]:
root = ElementTree.fromstring(r.content)

In [6]:
all_records = []
for i, child in enumerate(root):
    record = {}
    for subchild in child:
        record[subchild.tag] = subchild.text
    all_records.append(record)

In [7]:
df = pd.DataFrame(all_records)
# df = df.applymap(translate_to_western)
df

,date,dep,dis1,dis2,dis3,id,lat,long,mag,reg1,reg2,reg3
0,2017-12-24 12:54:52,10,3,7,9,130306,35.68 N,50.95 E,1.8,"Malard, Tehran","Meshkin dasht, Alborz","Safa dasht, Tehran"
1,2017-12-24 12:47:40,10,11,11,12,130305,35.60 N,50.89 E,1.8,"Malard, Tehran","Safa dasht, Tehran","Vahideieh, Tehran"
2,2017-12-24 12:39:21,10,4,7,8,130304,35.72 N,50.92 E,1.9,"Meshkin dasht, Alborz","Safa dasht, Tehran","Malard, Tehran"
3,2017-12-24 11:46:11,19,18,21,44,130303,31.24 N,56.22 E,2.0,"Kian shahr, kerman","Koohbanan, kerman","Yazdan shahr, kerman"
4,2017-12-24 11:13:33,12,18,25,27,130302,34.68 N,45.79 E,2.9,"Azgele, Kermanshah","Sar pol zahab, Kermanshah","Ghasre shirin, Kermanshah"
5,2017-12-24 11:08:34,8,1,30,52,130301,31.41 N,56.28 E,2.3,"Koohbanan, kerman","Kian shahr, kerman","Ravar, kerman"
6,2017-12-24 10:57:16,19,10,36,42,130300,27.80 N,53.43 E,3.9,"Khonj, Fars","Mahmeleh, Fars","Beiram, Fars"
7,2017-12-24 10:49:03,6,12,35,40,130299,35.70 N,52.88 E,1.9,"Firoozkooh, Tehran","Arjmand, Tehran","Sorkheh, Semnan"
8,2017-12-24 10:35:42,9,50,56,57,130298,32.05 N,55.47 E,2.0,"Bafgh, Yazd","Bahabad, Yazd","Saghand, Yazd"
9,2017-12-24 10:26:06,7,21,21,24,130297,32.11 N,51.43 E,1.6,"Borojen, Charmahal va Bakhtiari","Talkhoncheh, isfahan","Sefid dasht, Charmahal va Bakhtiari"


In [8]:
# Lets load bokeh!

from bokeh.io import output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, Range1d, PanTool, WheelZoomTool, BoxSelectTool
)

output_notebook()

Loading BokehJS ...

In [9]:
lats = df['lat'].apply(lambda x: x.split(' ')[0])
longs = df['long'].apply(lambda x: x.split(' ')[0])

In [15]:
# from Bokeh samples

map_options = GMapOptions(lat=35.68, lng=50.95, map_type="roadmap", zoom=10)

plot = GMapPlot(x_range=Range1d(), y_range=Range1d(), map_options=map_options)
plot.title.text = "Iran"

# For GMaps to function, Google requires you obtain and enable an API key:
#
#     https://developers.google.com/maps/documentation/javascript/get-api-key
#
# Replace the value below with your personal API key:
plot.api_key = "API-KEY-HERE"


source = ColumnDataSource(
    data=dict(
        lat=lats,
        lon=longs,
    )
)

circle = Circle(x="lon", y="lat", size=10, fill_color="red", fill_alpha=0.8, line_color=None)
plot.add_glyph(source, circle)

plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())

plot.width = 1280
plot.height = 720

show(plot)